<style>
body {
    max-width: 900px;
    margin: 40px auto;
    padding: 0 20px;
    font-family: "Georgia", serif;
    line-height: 1.6;
}
</style>

<div style="text-align: center; padding: 60px 60px">
  <h1 style="font-weight: bold; font-size: 3.1em">
    ENCONTRAR EL MEJOR ARIMA PARA LA SERIE DIFERENCIADA
  </h1>
</div>

Esta vez encontraremos un modelo variando la parte media móvil estacional y la parte autorregresiva estacional.

**RESULTADOS**:
- No hay modelos propuestos. Todos tienen dependencia en los residuos.
- `aic_values-4.csv`
- `AIC_3D_plot-4.html`.

# **CARGAR LOS DATOS**

In [1]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

data=pd.read_csv('MXN00021035.csv')

pre=data.iloc[:,6]  # Precipitacion, es la columna 5
date=data.iloc[:,5] # Date, es la columna 6
date = date.astype(str).str.replace(r'(\d{4})(\d{2})', r'\1/\2', regex=True)    # La fecha está como 195210 y la pasamos a 1952/10 
date = pd.to_datetime(date, format='%Y/%m')                                     # Lo convertimos en fecha
pre = pd.Series(pre.values, index=date)                                         # Creamos una Serie

# Partir la serie para train y test
pre_total = pre.copy()          # Copia de la serie original

# Todas hasta los ultimos 12 meses
pre = pre_total[:-12]           # Entrenamiento: todos menos los últimos 12 meses

from scipy.stats import zscore

# Estandarizar la serie original
zpre = zscore(pre)

# **BUSCAMOS EL MODELO CON MENOR $\text{AIC} $**

Se fue probando varias veces, y se buscaba ir encontrando el mejor $\text{AIC}$  en saltos, porque hacer todos los modelos tarda mucho tiempo. Por eso tiene mucho menos granulidad esta vez, que en la vez pasada.

In [61]:
def sarima(p, d, q, P, D, Q, s):    # La vamos a usar para calcular el AIC
    return SARIMAX(zpre, order=(p, d, q), seasonal_order=(P, D, Q, s)).fit(disp=0).aic    

Empezamos con el propuesto en 03

In [111]:
aics = []   # Lista para guardar los AICs

In [112]:
for P in range(0, 5):   # Para el componente estacional
    for Q in range(0, 5):   # Para el componente estacional
        aics.append((0, 0, 0, P, 1, Q, 12, sarima(0, 0, 0, P, 1, Q, 12)))

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date inde

In [113]:
sorted(aics, key=lambda x: x[7])

[(0, 0, 0, 0, 1, 3, 12, 1443.5046486930705),
 (0, 0, 0, 0, 1, 1, 12, 1443.5933641175748),
 (0, 0, 0, 2, 1, 1, 12, 1444.110845301806),
 (0, 0, 0, 1, 1, 2, 12, 1445.1467855295289),
 (0, 0, 0, 1, 1, 3, 12, 1445.2761261717749),
 (0, 0, 0, 0, 1, 4, 12, 1445.4876160065296),
 (0, 0, 0, 0, 1, 2, 12, 1445.5858681816667),
 (0, 0, 0, 1, 1, 1, 12, 1445.5874430840317),
 (0, 0, 0, 4, 1, 3, 12, 1445.703823120819),
 (0, 0, 0, 4, 1, 4, 12, 1445.7448183735353),
 (0, 0, 0, 2, 1, 2, 12, 1445.920682287981),
 (0, 0, 0, 4, 1, 2, 12, 1445.945951298871),
 (0, 0, 0, 3, 1, 1, 12, 1446.070360948445),
 (0, 0, 0, 2, 1, 3, 12, 1446.3837838663976),
 (0, 0, 0, 1, 1, 4, 12, 1446.56896129825),
 (0, 0, 0, 4, 1, 1, 12, 1446.6985652988965),
 (0, 0, 0, 3, 1, 2, 12, 1446.989869793793),
 (0, 0, 0, 3, 1, 3, 12, 1447.780088787008),
 (0, 0, 0, 2, 1, 4, 12, 1447.7999875463356),
 (0, 0, 0, 4, 1, 0, 12, 1448.2366123305535),
 (0, 0, 0, 3, 1, 4, 12, 1449.2145709622591),
 (0, 0, 0, 3, 1, 0, 12, 1454.1287421213299),
 (0, 0, 0, 2, 1, 0,

El modelo con menor $\text{AIC}$ es el $\text{ARIMA}(0,0,0) \times (0,1,3)$,  Con un $\text{AIC} = 1443.5046486930705$.

In [114]:
# Guardar los valores AIC en un archivo CSV
aic_df = pd.DataFrame(aics, columns=['p', 'd', 'q', 'P', 'D', 'Q', 's', 'AIC'])
aic_df.to_csv('aic_values-4.csv', index=False)

# **VISUALIZACIÓN**

In [115]:
import plotly.graph_objects as go
import numpy as np
from scipy.interpolate import griddata

In [116]:
# Separar en listas
p_values = [item[3] for item in aics]
q_values = [item[5] for item in aics]
aic_values = [item[7] for item in aics]

# Crear una malla regular
p_lin = np.linspace(min(p_values), max(p_values), 50)
q_lin = np.linspace(min(q_values), max(q_values), 50)
P, Q = np.meshgrid(p_lin, q_lin)

# Interpolar AIC en la malla
AIC_grid = griddata(
    points=(p_values, q_values),
    values=aic_values,
    xi=(P, Q),
    method='nearest'
)

In [117]:
# Crear figura 3D en Plotly
fig = go.Figure()

# Superficie
fig.add_trace(go.Surface(
    x=P, y=Q, z=AIC_grid,
    colorscale='Blues_r',
    opacity=0.7,
    colorbar=dict(title='AIC')
))

# Puntos originales
fig.add_trace(go.Scatter3d(
    x=p_values,
    y=q_values,
    z=aic_values,
    mode='markers',
    marker=dict(size=5, color=aic_values, colorscale='Blues_r'),
    name='Puntos'
))

# Layout
fig.update_layout(
    scene=dict(
        xaxis_title='P',
        yaxis_title='Q',
        zaxis_title='AIC'
    ),
    title='AIC para diferentes combinaciones de ARIMA(0,0,0)X(P,1,Q,12)',
    width=900,
    height=700
)

# Guardar como HTML
fig.write_html("AIC_3D_plot-4.html")
fig.show()

# **VERIFICACIÓN DE SUPUESTOS**

In [119]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from verificacion_de_supuestos import * # Una funcion que hice para verificar si los parametros son estacionarios/invertibles
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import ttest_1samp
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.tools.tools import add_constant
from scipy.stats import jarque_bera
from statsmodels.stats.diagnostic import lilliefors

## **MODELO 1**

In [120]:
modelo=SARIMAX(zpre,
               order=(0,0,0),
               seasonal_order=(0,1,3,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



<class 'statsmodels.iolib.summary.Summary'>
"""
                                    SARIMAX Results                                     
========================================================================================
Dep. Variable:                                y   No. Observations:                  660
Model:             SARIMAX(0, 1, [1, 2, 3], 12)   Log Likelihood                -717.752
Date:                          Sat, 26 Apr 2025   AIC                           1443.505
Time:                                  22:14:19   BIC                           1461.400
Sample:                                       0   HQIC                          1450.447
                                          - 660                                         
Covariance Type:                            opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.S.L12      -0.6576      0.029    -22.456      0.000      -0.715      -0.600
ma.S.L24      -0.0539      0.037     -1.461      0.144      -0.126       0.018
ma.S.L36       0.0890      0.034      2.655      0.008       0.023       0.155
sigma2         0.5308      0.019     27.564      0.000       0.493       0.569
===================================================================================
Ljung-Box (L1) (Q):                  14.53   Jarque-Bera (JB):               260.95
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.84   Skew:                             0.54
Prob(H) (two-sided):                  0.20   Kurtosis:                         5.92
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [121]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {}

El grado del polinomio es: 0

Raíces del polinomio característico: []

Módulo de las raíces: []

¿Las raíces están fuera del círculo unitario?  True

El modelo es estacionario

:)


In [122]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p12': -0.6576468674950458, 'p24': -0.05390724827273551, 'p36': 0.08902758389099319}

El grado del polinomio es: 36

Raíces del polinomio característico: [-1.05969906+0.28394551j -1.05969906-0.28394551j -1.0555843 +0.02843393j
 -1.0555843 -0.02843393j -0.92837979+0.50316764j -0.92837979-0.50316764j
 -0.89994585+0.55241666j -0.89994585-0.55241666j -0.77575355+0.77575355j
 -0.77575355-0.77575355j -0.55241666+0.89994585j -0.55241666-0.89994585j
 -0.50316764+0.92837979j -0.50316764-0.92837979j -0.28394551+1.05969906j
 -0.28394551-1.05969906j -0.02843393+1.0555843j  -0.02843393-1.0555843j
  0.02843393+1.0555843j   0.02843393-1.0555843j   0.28394551+1.05969906j
  0.28394551-1.05969906j  0.50316764+0.92837979j  0.50316764-0.92837979j
  0.55241666+0.89994585j  0.55241666-0.89994585j  0.77575355+0.77575355j
  0.77575355-0.77575355j  1.05969906+0.28394551j  1.05969906-0.28394551j
  1.0555843 +0.02843393j  1.0555843 -0.02843393j  0.89994585+0.55241666j
  

In [123]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,17.179415,NaN
2,18.181363,NaN
3,18.881778,NaN
4,25.921220,3.556381e-07
5,51.167881,7.745256e-12
6,70.743004,2.959018e-15
7,72.370244,7.167546e-15
8,73.200464,2.207929e-14
9,74.909882,4.005840e-14
10,80.168939,1.272448e-14


Hay dependencia en los residuos.

## **MODELO 2**

In [129]:
modelo=SARIMAX(zpre,
               order=(0,0,0),
               seasonal_order=(0,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



<class 'statsmodels.iolib.summary.Summary'>
"""
                                 SARIMAX Results                                  
==================================================================================
Dep. Variable:                          y   No. Observations:                  660
Model:             SARIMAX(0, 1, [1], 12)   Log Likelihood                -719.797
Date:                    Sat, 26 Apr 2025   AIC                           1443.593
Time:                            22:14:20   BIC                           1452.541
Sample:                                 0   HQIC                          1447.064
                                    - 660                                         
Covariance Type:                      opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.S.L12      -0.6533      0.025    -26.057      0.000      -0.702      -0.604
sigma2         0.5344      0.019     28.147      0.000       0.497       0.572
===================================================================================
Ljung-Box (L1) (Q):                  15.79   Jarque-Bera (JB):               286.71
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.83   Skew:                             0.59
Prob(H) (two-sided):                  0.17   Kurtosis:                         6.04
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [130]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {}

El grado del polinomio es: 0

Raíces del polinomio característico: []

Módulo de las raíces: []

¿Las raíces están fuera del círculo unitario?  True

El modelo es estacionario

:)


In [131]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p12': -0.6532809672286413}

El grado del polinomio es: 12

Raíces del polinomio característico: [-1.03611589e+00+0.j         -8.97302679e-01+0.51805794j
 -8.97302679e-01-0.51805794j -5.18057943e-01+0.89730268j
 -5.18057943e-01-0.89730268j  8.48820604e-17+1.03611589j
  8.48820604e-17-1.03611589j  5.18057943e-01+0.89730268j
  5.18057943e-01-0.89730268j  1.03611589e+00+0.j
  8.97302679e-01+0.51805794j  8.97302679e-01-0.51805794j]

Módulo de las raíces: [1.03611589 1.03611589 1.03611589 1.03611589 1.03611589 1.03611589
 1.03611589 1.03611589 1.03611589 1.03611589 1.03611589 1.03611589]

¿Las raíces están fuera del círculo unitario?  True

El modelo es invertible

:)


In [132]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,18.383847,NaN
2,19.527333,9.917048e-06
3,20.310714,3.886730e-05
4,27.528968,4.560262e-06
5,53.592080,6.406348e-11
6,73.402658,2.003677e-14
7,74.883915,4.055452e-14
8,75.550613,1.108538e-13
9,77.253111,1.743214e-13
10,83.190537,3.746241e-14


Hay dependencia en los residuos.

## **MODELO 3**

In [138]:
modelo=SARIMAX(zpre,
               order=(0,0,0),
               seasonal_order=(2,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



<class 'statsmodels.iolib.summary.Summary'>
"""
                                 SARIMAX Results                                  
==================================================================================
Dep. Variable:                          y   No. Observations:                  660
Model:             SARIMAX(2, 1, [1], 12)   Log Likelihood                -718.055
Date:                    Sat, 26 Apr 2025   AIC                           1444.111
Time:                            22:14:20   BIC                           1462.006
Sample:                                 0   HQIC                          1451.053
                                    - 660                                         
Covariance Type:                      opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.S.L12      -0.1041      0.069     -1.501      0.133      -0.240       0.032
ar.S.L24      -0.1141      0.050     -2.277      0.023      -0.212      -0.016
ma.S.L12      -0.5526      0.072     -7.689      0.000      -0.693      -0.412
sigma2         0.5313      0.019     27.725      0.000       0.494       0.569
===================================================================================
Ljung-Box (L1) (Q):                  14.85   Jarque-Bera (JB):               267.24
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.84   Skew:                             0.55
Prob(H) (two-sided):                  0.19   Kurtosis:                         5.95
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [139]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p12': 0.10410933674538352, 'p24': 0.11410119085946407}

El grado del polinomio es: 24

Raíces del polinomio característico: [-1.08336346+0.1568628j  -1.08336346-0.1568628j  -1.01665168+0.40583456j
 -1.01665168-0.40583456j -0.85978888+0.6775289j  -0.85978888-0.6775289j
 -0.6775289 +0.85978888j -0.6775289 -0.85978888j -0.40583456+1.01665168j
 -0.40583456-1.01665168j -0.1568628 +1.08336346j -0.1568628 -1.08336346j
  0.1568628 +1.08336346j  0.1568628 -1.08336346j  0.40583456+1.01665168j
  0.40583456-1.01665168j  0.6775289 +0.85978888j  0.6775289 -0.85978888j
  0.85978888+0.6775289j   0.85978888-0.6775289j   1.08336346+0.1568628j
  1.08336346-0.1568628j   1.01665168+0.40583456j  1.01665168-0.40583456j]

Módulo de las raíces: [1.09466082 1.09466082 1.09466082 1.09466082 1.09466082 1.09466082
 1.09466082 1.09466082 1.09466082 1.09466082 1.09466082 1.09466082
 1.09466082 1.09466082 1.09466082 1.09466082 1.09466082 1.09466082
 1.09466082 1.09466082 1

In [140]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p12': -0.5526053974957342}

El grado del polinomio es: 12

Raíces del polinomio característico: [-1.05066776e+00+0.j         -9.09904972e-01+0.52533388j
 -9.09904972e-01-0.52533388j -5.25333881e-01+0.90990497j
 -5.25333881e-01-0.90990497j -2.54511798e-16+1.05066776j
 -2.54511798e-16-1.05066776j  5.25333881e-01+0.90990497j
  5.25333881e-01-0.90990497j  1.05066776e+00+0.j
  9.09904972e-01+0.52533388j  9.09904972e-01-0.52533388j]

Módulo de las raíces: [1.05066776 1.05066776 1.05066776 1.05066776 1.05066776 1.05066776
 1.05066776 1.05066776 1.05066776 1.05066776 1.05066776 1.05066776]

¿Las raíces están fuera del círculo unitario?  True

El modelo es invertible

:)


In [141]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,17.493428,NaN
2,18.519956,NaN
3,19.228301,NaN
4,26.315332,2.899775e-07
5,51.914435,5.332412e-12
6,71.654631,1.887537e-15
7,73.318872,4.517343e-15
8,74.131900,1.411701e-14
9,75.884632,2.523257e-14
10,81.283735,7.536739e-15


Hay dependencia en los residuos.

## **MODELO 4**

In [147]:
modelo=SARIMAX(zpre,
               order=(0,0,0),
               seasonal_order=(1,1,2,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



<class 'statsmodels.iolib.summary.Summary'>
"""
                                   SARIMAX Results                                   
=====================================================================================
Dep. Variable:                             y   No. Observations:                  660
Model:             SARIMAX(1, 1, [1, 2], 12)   Log Likelihood                -718.573
Date:                       Sat, 26 Apr 2025   AIC                           1445.147
Time:                               22:14:22   BIC                           1463.042
Sample:                                    0   HQIC                          1452.089
                                       - 660                                         
Covariance Type:                         opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.S.L12      -0.8935      0.138     -6.496      0.000      -1.163      -0.624
ma.S.L12       0.2628      0.132      1.985      0.047       0.003       0.522
ma.S.L24      -0.6117      0.077     -7.912      0.000      -0.763      -0.460
sigma2         0.5321      0.019     28.185      0.000       0.495       0.569
===================================================================================
Ljung-Box (L1) (Q):                  14.91   Jarque-Bera (JB):               285.36
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.84   Skew:                             0.57
Prob(H) (two-sided):                  0.20   Kurtosis:                         6.05
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [148]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p12': 0.8934837600190328}

El grado del polinomio es: 12

Raíces del polinomio característico: [-0.97503429+0.26125965j -0.97503429-0.26125965j -0.71377464+0.71377464j
 -0.71377464-0.71377464j -0.26125965+0.97503429j -0.26125965-0.97503429j
  0.26125965+0.97503429j  0.26125965-0.97503429j  0.71377464+0.71377464j
  0.71377464-0.71377464j  0.97503429+0.26125965j  0.97503429-0.26125965j]

Módulo de las raíces: [1.00942978 1.00942978 1.00942978 1.00942978 1.00942978 1.00942978
 1.00942978 1.00942978 1.00942978 1.00942978 1.00942978 1.00942978]

¿Las raíces están fuera del círculo unitario?  True

El modelo es estacionario

:)


In [149]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p12': 0.2628444255001554, 'p24': -0.6117107674153909}

El grado del polinomio es: 24

Raíces del polinomio característico: [-1.03501581e+00+0.j         -9.72264921e-01+0.2605176j
 -9.72264921e-01-0.2605176j  -8.96349985e-01+0.51750791j
 -8.96349985e-01-0.51750791j -7.11747321e-01+0.71174732j
 -7.11747321e-01-0.71174732j -5.17507905e-01+0.89634999j
 -5.17507905e-01-0.89634999j -2.60517601e-01+0.97226492j
 -2.60517601e-01-0.97226492j -4.10915192e-16+1.03501581j
 -4.10915192e-16-1.03501581j  2.60517601e-01+0.97226492j
  2.60517601e-01-0.97226492j  5.17507905e-01+0.89634999j
  5.17507905e-01-0.89634999j  7.11747321e-01+0.71174732j
  7.11747321e-01-0.71174732j  1.03501581e+00+0.j
  9.72264921e-01+0.2605176j   9.72264921e-01-0.2605176j
  8.96349985e-01+0.51750791j  8.96349985e-01-0.51750791j]

Módulo de las raíces: [1.03501581 1.00656271 1.00656271 1.03501581 1.03501581 1.00656271
 1.00656271 1.03501581 1.03501581 1.00656271 1.00656271 1.03501581
 1

In [150]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,17.463603,NaN
2,18.645709,NaN
3,19.472263,NaN
4,26.854810,2.193278e-07
5,52.510958,3.957221e-12
6,72.359787,1.333039e-15
7,74.034847,3.188012e-15
8,74.653056,1.099051e-14
9,76.187459,2.185605e-14
10,82.058149,5.236751e-15


Hay dependencia en los residuos.

## **MODELO 5**

In [156]:
modelo=SARIMAX(zpre,
               order=(1,0,3),
               seasonal_order=(0,1,4,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



<class 'statsmodels.iolib.summary.Summary'>
"""
                                           SARIMAX Results                                           
=====================================================================================================
Dep. Variable:                                             y   No. Observations:                  660
Model:             SARIMAX(1, 0, 3)x(0, 1, [1, 2, 3, 4], 12)   Log Likelihood                -707.513
Date:                                       Sat, 26 Apr 2025   AIC                           1433.026
Time:                                               22:14:51   BIC                           1473.291
Sample:                                                    0   HQIC                          1448.646
                                                       - 660                                         
Covariance Type:                                         opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.1204      0.438     -0.275      0.783      -0.978       0.738
ma.L1          0.2845      0.436      0.652      0.514      -0.571       1.140
ma.L2          0.0874      0.080      1.090      0.276      -0.070       0.245
ma.L3          0.1134      0.044      2.548      0.011       0.026       0.201
ma.S.L12      -0.6893      0.029    -23.542      0.000      -0.747      -0.632
ma.S.L24      -0.0505      0.037     -1.366      0.172      -0.123       0.022
ma.S.L36       0.0709      0.038      1.846      0.065      -0.004       0.146
ma.S.L48      -0.0032      0.034     -0.093      0.926      -0.070       0.064
sigma2         0.5135      0.020     25.670      0.000       0.474       0.553
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               207.43
Prob(Q):                              0.98   Prob(JB):                         0.00
Heteroskedasticity (H):               0.86   Skew:                             0.44
Prob(H) (two-sided):                  0.25   Kurtosis:                         5.63
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [157]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p1': 0.12042050604759126}

El grado del polinomio es: 1

Raíces del polinomio característico: [-8.3042335]

Módulo de las raíces: [8.3042335]

¿Las raíces están fuera del círculo unitario?  True

El modelo es estacionario

:)


In [158]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': 0.28453739913448367, 'p2': 0.08744263796213617, 'p3': 0.11336638804641153, 'p12': -0.689349750524581, 'p24': -0.050470186215013375, 'p36': 0.0708803762819837, 'p48': -0.0031726073065468717}

El grado del polinomio es: 48

Raíces del polinomio característico: [ 1.28962843e+00+0.j          1.11674298e+00+0.64486861j
  1.11674298e+00-0.64486861j  1.08048252e+00+0.04326004j
  1.08048252e+00-0.04326004j  1.06965373e+00+0.28919708j
  1.06965373e+00-0.28919708j  6.44662256e-01+1.11675118j
  6.44662256e-01-1.11675118j  9.63397802e-01+0.52088179j
  9.63397802e-01-0.52088179j  8.91652798e-01+0.57127551j
  8.91652798e-01-0.57127551j  7.77061882e-01+0.78436918j
  7.77061882e-01-0.78436918j -2.28099765e-05+1.28946341j
 -2.28099765e-05-1.28946341j  5.64659117e-01+0.94385916j
  5.64659117e-01-0.94385916j  4.83589283e-01+0.91585203j
  4.83589283e-01-0.91585203j  2.81736324e-01+1.0642343j
  2.81736324e-01-1.0642343j   8.52825570e-03+1.09648617j
  8.528255

In [159]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.127577,NaN
2,0.232730,NaN
3,0.773182,NaN
4,2.709120,NaN
5,20.690359,NaN
6,35.530901,NaN
7,35.928114,NaN
8,36.837023,NaN
9,38.638278,5.100869e-10
10,41.799973,8.380137e-10


Hay dependencia en los residuos.

## **MODELO 6**

In [165]:
modelo=SARIMAX(zpre,
               order=(0,0,0),
               seasonal_order=(0,1,2,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



<class 'statsmodels.iolib.summary.Summary'>
"""
                                   SARIMAX Results                                   
=====================================================================================
Dep. Variable:                             y   No. Observations:                  660
Model:             SARIMAX(0, 1, [1, 2], 12)   Log Likelihood                -719.793
Date:                       Sat, 26 Apr 2025   AIC                           1445.586
Time:                               22:14:52   BIC                           1459.008
Sample:                                    0   HQIC                          1450.793
                                       - 660                                         
Covariance Type:                         opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.S.L12      -0.6560      0.029    -22.764      0.000      -0.712      -0.599
ma.S.L24       0.0038      0.031      0.122      0.903      -0.057       0.064
sigma2         0.5344      0.019     27.963      0.000       0.497       0.572
===================================================================================
Ljung-Box (L1) (Q):                  15.75   Jarque-Bera (JB):               285.46
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.83   Skew:                             0.59
Prob(H) (two-sided):                  0.18   Kurtosis:                         6.03
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [166]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {}

El grado del polinomio es: 0

Raíces del polinomio característico: []

Módulo de las raíces: []

¿Las raíces están fuera del círculo unitario?  True

El modelo es estacionario

:)


In [167]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p12': -0.6559709015852816, 'p24': 0.0037660425032464548}

El grado del polinomio es: 24

Raíces del polinomio característico: [-1.53613348e+00+0.j         -1.33033061e+00+0.76806674j
 -1.33033061e+00-0.76806674j -7.68066738e-01+1.33033061j
 -7.68066738e-01-1.33033061j -9.85778834e-16+1.53613348j
 -9.85778834e-16-1.53613348j  7.68066738e-01+1.33033061j
  7.68066738e-01-1.33033061j  1.33033061e+00+0.76806674j
  1.33033061e+00-0.76806674j  1.53613348e+00+0.j
 -1.03652698e+00+0.j         -8.97658698e-01+0.51826349j
 -8.97658698e-01-0.51826349j -5.18263491e-01+0.8976587j
 -5.18263491e-01-0.8976587j  -1.05918390e-16+1.03652698j
 -1.05918390e-16-1.03652698j  5.18263491e-01+0.8976587j
  5.18263491e-01-0.8976587j   8.97658698e-01+0.51826349j
  8.97658698e-01-0.51826349j  1.03652698e+00+0.j        ]

Módulo de las raíces: [1.53613348 1.53613348 1.53613348 1.53613348 1.53613348 1.53613348
 1.53613348 1.53613348 1.53613348 1.53613348 1.53613348 1.53613348

In [168]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,18.349617,NaN
2,19.494151,NaN
3,20.284211,6.674915e-06
4,27.484417,1.076056e-06
5,53.552157,1.398039e-11
6,73.369953,4.406415e-15
7,74.849287,1.000165e-14
8,75.515860,3.005519e-14
9,77.220547,5.072682e-14
10,83.141255,1.137665e-14


Hay dependencia en los residuos.